In [155]:
import asyncio
import pybotters
import pandas as pd
from loguru import logger

import pandas as pd
import datetime
import math
from typing import Tuple

In [156]:
# e.g. https://api.cryptowat.ch/markets/:exchange/:pair/ohlc
def generate_path(
        exchange: str,
        pair: str,
        min: int,
        unixTime: int) -> str:
    
    return f'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc?periods={min}&after={unixTime}'

print(generate_path('bitflyer', 'btcfxjpy', 900, 1675368565))

https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=900&after=1675368565


In [157]:
def str_to_timestamp(s: str) -> int:

    # str → datetime e.g.'2022-05-27 10:11:23'
    dt = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

    # datetime → timestamp
    ts = datetime.datetime.timestamp(dt)

    return math.floor(ts)

ts = str_to_timestamp('2022-05-27 10:11:23')
print(ts)

1653613883


In [158]:
def get_url(candle: int, date: str, size: int) -> Tuple[str, str]:
        url = 'https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods='
        candle = candle*60
        period = ('%s'%(candle))
        beforeurl = '&before='
        afterurl = '&after='
        # The maximum size of data is 6000 depending on a timing (sometimes you get less than 6000)
        before = str_to_timestamp(date)
        after = before - (candle * size)
        URL=('%s%s%s%s%s%s'%(url, candle, afterurl, after, beforeurl, before))
        # e.g https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1483196400

        return URL, period

In [159]:
async def main(candle: int, date: str, size: int) -> pd.DataFrame:
    async with pybotters.Client() as client:
        URL, period = get_url(candle, date, size)

        response = await client.get(URL)
        body = await response.json()
        res = body['result']
        
        data = []
        for column in res[period]:
            if column[4] != 0:
                column = column[0:6]
                data.append(column)

        date = [price[0] for price in data]
        op = [float(price[1]) for price in data]
        hi = [float(price[2]) for price in data]
        lo = [float(price[3]) for price in data]
        cl = [float(price[4]) for price in data]
        volume = [float(price[5]) for price in data]
        date_datetime = map(datetime.datetime.fromtimestamp, date)
        dti = pd.DatetimeIndex(date_datetime)
        df = pd.DataFrame({"op" : op, "hi" : hi, "lo": lo, "cl" : cl, "volume" : volume}, index=dti)
        
        return df

In [160]:
if __name__ == '__main__':
    try:
        df = await main(15, '2023-02-02 20:09:25', 6000)
        display(df)
        print(df.dtypes)
        df.to_pickle('df_ohlcv.pkl')
    except KeyboardInterrupt:
        pass

,op,hi,lo,cl,volume
2022-12-03 18:00:00,2295342.0,2298682.0,2293000.0,2295615.0,77.574174
2022-12-03 18:15:00,2295696.0,2298321.0,2294116.0,2297404.0,30.543160
2022-12-03 18:30:00,2297487.0,2298157.0,2296549.0,2298122.0,8.825385
2022-12-03 18:45:00,2298102.0,2298913.0,2297168.0,2297429.0,13.106698
2022-12-03 19:00:00,2297695.0,2300840.0,2297424.0,2300109.0,28.126075
...,...,...,...,...,...
2023-02-02 19:00:00,3076436.0,3080258.0,3074061.0,3079758.0,38.497302
2023-02-02 19:15:00,3079612.0,3080461.0,3076000.0,3077042.0,26.473347
2023-02-02 19:30:00,3076671.0,3078445.0,3075097.0,3076117.0,22.083219
2023-02-02 19:45:00,3076233.0,3076907.0,3071484.0,3074388.0,27.569929


op        float64
hi        float64
lo        float64
cl        float64
volume    float64
dtype: object
